<a href="https://colab.research.google.com/github/oaarnikoivu/dissertation/blob/master/Emotional_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Detecting Emotions from Tweets with BERT 

First we need to install the transformers python package to get access to the pre-trained BERT models.

In [0]:
!pip install transformers
!pip install fast-bert

In [0]:
from transformers import BertTokenizer, BertConfig, BertForSequenceClassification
from box import Box
from tqdm import tqdm, trange
from pathlib import Path
from sklearn.model_selection import train_test_split

import torch
import pandas as pd 
import collections 
import random
import numpy as np

import logging
import datetime

from fast_bert.modeling import BertForMultiLabelSequenceClassification
from fast_bert.data_cls import BertDataBunch, InputExample, InputFeatures, MultiLabelTextProcessor, convert_examples_to_features
from fast_bert.learner_cls import BertLearner
from fast_bert.metrics import accuracy_multilabel, accuracy_thresh, fbeta, roc_auc

In [44]:
from google.colab import drive 
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
logging.basicConfig(format='%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)
logger = logging.getLogger(__name__)

In [0]:
torch.cuda.empty_cache()

In [0]:
run_start_time = datetime.datetime.today().strftime('%Y-%m-%d_%H-%M-%S')

In [0]:
path = '/content/drive/My Drive/'

DATA_PATH = Path(path + '/datasets/SemEval/')
LABEL_PATH = Path(path + '/labels/')

MODEL_PATH = Path(path + '/models/')
LOG_PATH = Path(path + '/logs/')

MODEL_PATH.mkdir(exist_ok=True)
LOG_PATH.mkdir(exist_ok=True)

BERT_PRETRAINED_PATH = Path(path + '/BERT/')

OUTPUT_PATH = Path(MODEL_PATH/'output')
OUTPUT_PATH.mkdir(exist_ok=True)

# BERT Arguments

In [0]:
args = Box({
    "multi_gpu": False,
    "multi_label": True,
    "batch_size_per_gpu": 16,
    "warmup_steps": 500,
    "max_seq_length": 512,
    "num_epochs": 6, 
    "is_fp16": False,
    "logging_steps": 50,
    "schedule_type": 'warmup_cosine',
    "optimizer_type": 'lamb',
    "learning_rate": 6e-5
})

# GPU

In [0]:
device = torch.device('cuda')

# check if multiple GPUs are available

if torch.cuda.device_count() > 1:
    args.multi_gpu = True
else:
    args.multi_gpu = False

# Custom BERT Models

In [0]:
class MyMultiLabelBertClassifier(BertForSequenceClassification):
  """Bert model for classification."""

  def forward(
      self,
      input_ids,
      token_type_ids=None,
      attention_mask=None,
      labels=None,
      position_ids=None,
      head_mask=None,
  ):

    outputs = self.bert(
        input_ids, 
        position_ids=position_ids,
        token_type_ids=token_type_ids,
        attention_mask=attention_mask,
        head_mask=head_mask
    )
    pooled_output = outputs[1]

    pooled_output = self.dropout(pooled_output)
    logits = self.classifier(pooled_output)

    # add hidden states and attention if they are here
    outputs = (logits,) + outputs[2:]

    if labels is not None:
      loss_fct = BCEWithLogitsLoss()

      loss = loss_fct(
          logits.view(-1, self.num_labels), labels_view(-1, self.num_labels)
      )
      outputs = (loss,) + outputs
    
    return outputs 

In [0]:
label_cols = ['anger', 'anticipation', 'disgust', 'fear', 'joy', 
              'love', 'optimism', 'pessimism', 'sadness', 'surprise', 'trust']

# FastBert DataBunch Object

In [118]:
model = MultiLabelBertClassifier(config=BertConfig.from_pretrained('bert-base-uncased'))

databunch = BertDataBunch(DATA_PATH, 
                          LABEL_PATH,
                          tokenizer='bert-base-uncased',
                          train_file='train.csv',
                          val_file='val.csv',
                          test_data='test.csv',
                          label_file='labels.csv',
                          text_col="Tweet",
                          label_col=label_cols,
                          batch_size_per_gpu=args.batch_size_per_gpu,
                          max_seq_length=args.max_seq_length,
                          multi_gpu=args.multi_gpu,
                          multi_label=True,
                          model_type=model)

01/17/2020 13:34:42 - INFO - transformers.configuration_utils -   loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /root/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.bf3b9ea126d8c0001ee8a1e8b92229871d06d36d8808208cc2449280da87785c
01/17/2020 13:34:42 - INFO - transformers.configuration_utils -   Model config {
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "prun

KeyError: ignored

In [119]:
databunch.train_dl.dataset[0][3]

tensor([0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 1.])

In [120]:
num_labels = len(databunch.labels)
num_labels

11

# Evaluation Metrics

In [0]:
metrics = []
metrics.append({'name': 'accuracy_thresh', 'function': accuracy_thresh})
metrics.append({'name': 'roc_auc', 'function': roc_auc})
metrics.append({'name': 'fbeta', 'function': fbeta})

# Define FastBert Learner

In [122]:
learner = BertLearner.from_pretrained_model(
    databunch,
    pretrained_path='bert-base-uncased',
    metrics=metrics,
    device=device,
    logger=logger,
    output_dir=OUTPUT_PATH,
    finetuned_wgts_path=None,
    warmup_steps=args.warmup_steps,
    multi_gpu=args.multi_gpu,
    is_fp16=args.is_fp16,
    multi_label=args.multi_label,
    logging_steps=args.logging_steps)

01/17/2020 13:34:56 - INFO - transformers.configuration_utils -   loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /root/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.bf3b9ea126d8c0001ee8a1e8b92229871d06d36d8808208cc2449280da87785c
01/17/2020 13:34:56 - INFO - transformers.configuration_utils -   Model config {
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 11,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pru

bert-base-uncased
<class 'str'>


01/17/2020 13:34:56 - INFO - transformers.modeling_utils -   loading weights file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-pytorch_model.bin from cache at /root/.cache/torch/transformers/aa1ef1aede4482d0dbcd4d52baad8ae300e60902e88fcb0bebdec09afd232066.36ca03ab34a1a5d5fa7bc3d03d55c4fa650fed07220e2eeebc06ce58d0e9a157
01/17/2020 13:34:59 - INFO - transformers.modeling_utils -   Weights of BertForMultiLabelSequenceClassification not initialized from pretrained model: ['classifier.weight', 'classifier.bias']
01/17/2020 13:34:59 - INFO - transformers.modeling_utils -   Weights from pretrained model not used in BertForMultiLabelSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']


# Train the model

In [123]:
learner.fit(args.num_epochs, 
            args.learning_rate, 
            schedule_type=args.schedule_type,
            optimizer_type=args.optimizer_type,
            validate=True)

01/17/2020 13:35:10 - INFO - __main__ -   ***** Running training *****
01/17/2020 13:35:10 - INFO - __main__ -     Num examples = 6838
01/17/2020 13:35:10 - INFO - __main__ -     Num Epochs = 6
01/17/2020 13:35:10 - INFO - __main__ -     Total train batch size (w. parallel, distributed & accumulation) = 16
01/17/2020 13:35:10 - INFO - __main__ -     Gradient Accumulation steps = 1
01/17/2020 13:35:10 - INFO - __main__ -     Total optimization steps = 2568


01/17/2020 13:35:25 - INFO - __main__ -   Running evaluation
01/17/2020 13:35:25 - INFO - __main__ -     Num examples = 886
01/17/2020 13:35:25 - INFO - __main__ -     Batch size = 32


01/17/2020 13:35:28 - INFO - __main__ -   eval_loss after step 50: 0.67385775915214: 
01/17/2020 13:35:28 - INFO - __main__ -   eval_accuracy_thresh after step 50: 0.5244202613830566: 
01/17/2020 13:35:28 - INFO - __main__ -   eval_roc_auc after step 50: 0.5732082823786078: 
01/17/2020 13:35:28 - INFO - __main__ -   eval_fbeta after step 50: 0.5624696612358093: 
01/17/2020 13:35:28 - INFO - __main__ -   lr after step 50: 6e-06
01/17/2020 13:35:28 - INFO - __main__ -   train_loss after step 50: 0.6821628642082215
01/17/2020 13:35:43 - INFO - __main__ -   Running evaluation
01/17/2020 13:35:43 - INFO - __main__ -     Num examples = 886
01/17/2020 13:35:43 - INFO - __main__ -     Batch size = 32


01/17/2020 13:35:47 - INFO - __main__ -   eval_loss after step 100: 0.6644594669342041: 
01/17/2020 13:35:47 - INFO - __main__ -   eval_accuracy_thresh after step 100: 0.5532525777816772: 
01/17/2020 13:35:47 - INFO - __main__ -   eval_roc_auc after step 100: 0.6006698722514572: 
01/17/2020 13:35:47 - INFO - __main__ -   eval_fbeta after step 100: 0.5624696612358093: 
01/17/2020 13:35:47 - INFO - __main__ -   lr after step 100: 1.2e-05
01/17/2020 13:35:47 - INFO - __main__ -   train_loss after step 100: 0.6790372478961945
01/17/2020 13:36:01 - INFO - __main__ -   Running evaluation
01/17/2020 13:36:01 - INFO - __main__ -     Num examples = 886
01/17/2020 13:36:01 - INFO - __main__ -     Batch size = 32


01/17/2020 13:36:05 - INFO - __main__ -   eval_loss after step 150: 0.6521050057240895: 
01/17/2020 13:36:05 - INFO - __main__ -   eval_accuracy_thresh after step 150: 0.6243587136268616: 
01/17/2020 13:36:05 - INFO - __main__ -   eval_roc_auc after step 150: 0.60626730160144: 
01/17/2020 13:36:05 - INFO - __main__ -   eval_fbeta after step 150: 0.5624696612358093: 
01/17/2020 13:36:05 - INFO - __main__ -   lr after step 150: 1.8e-05
01/17/2020 13:36:05 - INFO - __main__ -   train_loss after step 150: 0.6679023969173431
01/17/2020 13:36:19 - INFO - __main__ -   Running evaluation
01/17/2020 13:36:19 - INFO - __main__ -     Num examples = 886
01/17/2020 13:36:19 - INFO - __main__ -     Batch size = 32


...

01/17/2020 13:50:08 - INFO - __main__ -   eval_loss after step 2450: 0.3729876471417291: 
01/17/2020 13:50:08 - INFO - __main__ -   eval_accuracy_thresh after step 2450: 0.8475271463394165: 
01/17/2020 13:50:08 - INFO - __main__ -   eval_roc_auc after step 2450: 0.8584200593184691: 
01/17/2020 13:50:08 - INFO - __main__ -   eval_fbeta after step 2450: 0.6467469334602356: 
01/17/2020 13:50:08 - INFO - __main__ -   lr after step 2450: 4.80718323058289e-07
01/17/2020 13:50:08 - INFO - __main__ -   train_loss after step 2450: 0.3788866072893143
01/17/2020 13:50:22 - INFO - __main__ -   Running evaluation
01/17/2020 13:50:22 - INFO - __main__ -     Num examples = 886
01/17/2020 13:50:22 - INFO - __main__ -     Batch size = 32


01/17/2020 13:50:26 - INFO - __main__ -   eval_loss after step 2500: 0.37297941957201275: 
01/17/2020 13:50:26 - INFO - __main__ -   eval_accuracy_thresh after step 2500: 0.8477323651313782: 
01/17/2020 13:50:26 - INFO - __main__ -   eval_roc_auc after step 2500: 0.8584259771334409: 
01/17/2020 13:50:26 - INFO - __main__ -   eval_fbeta after step 2500: 0.6476169228553772: 
01/17/2020 13:50:26 - INFO - __main__ -   lr after step 2500: 1.5992690042286096e-07
01/17/2020 13:50:26 - INFO - __main__ -   train_loss after step 2500: 0.3844040960073471
01/17/2020 13:50:40 - INFO - __main__ -   Running evaluation
01/17/2020 13:50:40 - INFO - __main__ -     Num examples = 886
01/17/2020 13:50:40 - INFO - __main__ -     Batch size = 32


01/17/2020 13:50:44 - INFO - __main__ -   eval_loss after step 2550: 0.3729843965598515: 
01/17/2020 13:50:44 - INFO - __main__ -   eval_accuracy_thresh after step 2550: 0.8477323651313782: 
01/17/2020 13:50:44 - INFO - __main__ -   eval_roc_auc after step 2550: 0.8584292410829357: 
01/17/2020 13:50:44 - INFO - __main__ -   eval_fbeta after step 2550: 0.647585928440094: 
01/17/2020 13:50:44 - INFO - __main__ -   lr after step 2550: 1.1215222253061485e-08
01/17/2020 13:50:44 - INFO - __main__ -   train_loss after step 2550: 0.36757662773132327
01/17/2020 13:50:49 - INFO - __main__ -   Running evaluation
01/17/2020 13:50:49 - INFO - __main__ -     Num examples = 886
01/17/2020 13:50:49 - INFO - __main__ -     Batch size = 32


01/17/2020 13:50:53 - INFO - __main__ -   eval_loss after epoch 6: 0.37298436037131716: 
01/17/2020 13:50:53 - INFO - __main__ -   eval_accuracy_thresh after epoch 6: 0.8477323651313782: 
01/17/2020 13:50:53 - INFO - __main__ -   eval_roc_auc after epoch 6: 0.8584293936039402: 
01/17/2020 13:50:53 - INFO - __main__ -   eval_fbeta after epoch 6: 0.647585928440094: 
01/17/2020 13:50:53 - INFO - __main__ -   lr after epoch 6: 0.0
01/17/2020 13:50:53 - INFO - __main__ -   train_loss after epoch 6: 0.3764647203628148
01/17/2020 13:50:53 - INFO - __main__ -   



(2568, 0.45126586347075637)

# Save Model 

In [124]:
learner.save_model()

01/17/2020 13:51:35 - INFO - transformers.configuration_utils -   Configuration saved in /content/drive/My Drive/models/output/model_out/config.json
01/17/2020 13:51:37 - INFO - transformers.modeling_utils -   Model weights saved in /content/drive/My Drive/models/output/model_out/pytorch_model.bin


# Inference

In [0]:
from fast_bert.prediction import BertClassificationPredictor

In [126]:
MODEL_PATH = OUTPUT_PATH/'model_out'

predictor = BertClassificationPredictor(model_path=MODEL_PATH, 
                                        label_path=LABEL_PATH,
                                        multi_label=True,
                                        model_type='bert',
                                        do_lower_case=True)

01/17/2020 13:51:40 - INFO - transformers.tokenization_utils -   Model name '/content/drive/My Drive/models/output/model_out' not found in model shortcut name list (bert-base-uncased, bert-large-uncased, bert-base-cased, bert-large-cased, bert-base-multilingual-uncased, bert-base-multilingual-cased, bert-base-chinese, bert-base-german-cased, bert-large-uncased-whole-word-masking, bert-large-cased-whole-word-masking, bert-large-uncased-whole-word-masking-finetuned-squad, bert-large-cased-whole-word-masking-finetuned-squad, bert-base-cased-finetuned-mrpc, bert-base-german-dbmdz-cased, bert-base-german-dbmdz-uncased, bert-base-finnish-cased-v1, bert-base-finnish-uncased-v1). Assuming '/content/drive/My Drive/models/output/model_out' is a path or url to a directory containing tokenizer files.
01/17/2020 13:51:40 - INFO - transformers.tokenization_utils -   loading file /content/drive/My Drive/models/output/model_out/vocab.txt
01/17/2020 13:51:40 - INFO - transformers.tokenization_utils -  

/content/drive/My Drive/models/output/model_out
<class 'str'>


In [134]:
single_pred = predictor.predict("Such a gloomy day, I honestly don't what to do with mysel. #bored")

01/17/2020 13:52:53 - INFO - root -   Writing example 0 of 1


In [135]:
single_pred

[('sadness', 0.5184949040412903),
 ('anger', 0.45845359563827515),
 ('disgust', 0.4444648027420044),
 ('fear', 0.3596719205379486),
 ('pessimism', 0.2696155905723572),
 ('optimism', 0.1702408343553543),
 ('anticipation', 0.1692613959312439),
 ('joy', 0.13945797085762024),
 ('trust', 0.11005723476409912),
 ('surprise', 0.10396468639373779),
 ('love', 0.08772367238998413)]